<a href="https://colab.research.google.com/github/vinu139/bvvs/blob/main/code_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import tkinter as tk
from tkinter import ttk, messagebox

# ------------------ DATABASE SETUP ------------------ #
conn = sqlite3.connect("airplanes.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS airplanes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    airline TEXT NOT NULL,
    model TEXT NOT NULL,
    capacity INTEGER NOT NULL,
    year INTEGER NOT NULL,
    color TEXT NOT NULL
)
""")
conn.commit()

# ------------------ FUNCTIONS ------------------ #
def add_airplane():
    try:
        airline = airline_entry.get()
        model = model_entry.get()
        capacity = int(capacity_entry.get())
        year = int(year_entry.get())
        color = color_var.get()

        if not airline or not model:
            raise ValueError("Airline and Model are required")

        cursor.execute("INSERT INTO airplanes (airline, model, capacity, year, color) VALUES (?, ?, ?, ?, ?)",
                       (airline, model, capacity, year, color))
        conn.commit()
        messagebox.showinfo("Success", "Airplane added successfully!")
        clear_inputs()
        view_airplanes()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def update_airplane():
    try:
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Warning", "Select a row to update")
            return

        row_id = tree.item(selected)['values'][0]
        airline = airline_entry.get()
        model = model_entry.get()
        capacity = int(capacity_entry.get())
        year = int(year_entry.get())
        color = color_var.get()

        cursor.execute("""
            UPDATE airplanes SET airline=?, model=?, capacity=?, year=?, color=?
            WHERE id=?
        """, (airline, model, capacity, year, color, row_id))
        conn.commit()
        messagebox.showinfo("Updated", "Airplane updated successfully!")
        clear_inputs()
        view_airplanes()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def delete_airplane():
    selected = tree.selection()
    if not selected:
        messagebox.showwarning("Warning", "Select a row to delete")
        return

    row_id = tree.item(selected)['values'][0]
    confirm = messagebox.askyesno("Confirm", "Are you sure you want to delete this airplane?")
    if confirm:
        cursor.execute("DELETE FROM airplanes WHERE id=?", (row_id,))
        conn.commit()
        messagebox.showinfo("Deleted", "Airplane deleted successfully!")
        clear_inputs()
        view_airplanes()

def view_airplanes():
    for row in tree.get_children():
        tree.delete(row)

    cursor.execute("SELECT * FROM airplanes WHERE year BETWEEN 1980 AND 2035")
    rows = cursor.fetchall()
    for row in rows:
        tree.insert("", tk.END, values=row)

def clear_inputs():
    airline_entry.delete(0, tk.END)
    model_entry.delete(0, tk.END)
    capacity_entry.delete(0, tk.END)
    year_entry.delete(0, tk.END)
    color_var.set("White")

def on_tree_select(event):
    selected = tree.selection()
    if selected:
        row = tree.item(selected)["values"]
        airline_entry.delete(0, tk.END)
        airline_entry.insert(0, row[1])
        model_entry.delete(0, tk.END)
        model_entry.insert(0, row[2])
        capacity_entry.delete(0, tk.END)
        capacity_entry.insert(0, row[3])
        year_entry.delete(0, tk.END)
        year_entry.insert(0, row[4])
        color_var.set(row[5])

# ------------------ THEME MANAGEMENT ------------------ #
def apply_theme(theme):
    colors = {
        "light": {"bg": "white", "fg": "black"},
        "dark": {"bg": "#2e2e2e", "fg": "white"},
        "rgb": {"bg": "#1e1f26", "fg": "#00ffff"},
    }

    c = colors[theme]
    root.configure(bg=c["bg"])
    input_frame.configure(bg=c["bg"])
    display_frame.configure(bg=c["bg"])
    theme_frame.configure(bg=c["bg"])

    for widget in input_frame.winfo_children() + theme_frame.winfo_children():
        if isinstance(widget, (tk.Label, tk.Button, ttk.Combobox, tk.Entry)):
            try:
                widget.configure(bg=c["bg"], fg=c["fg"])
            except:
                pass

# ------------------ GUI SETUP ------------------ #
root = tk.Tk()
root.title("Airplane Inventory Manager")
root.geometry("850x600")

# --------- Input Frame --------- #
input_frame = tk.LabelFrame(root, text="Manage Airplane", padx=10, pady=10)
input_frame.pack(fill="x", padx=10, pady=10)

tk.Label(input_frame, text="Airline:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
airline_entry = tk.Entry(input_frame)
airline_entry.grid(row=0, column=1, padx=5, pady=5)

tk.Label(input_frame, text="Model:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
model_entry = tk.Entry(input_frame)
model_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Label(input_frame, text="Capacity:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
capacity_entry = tk.Entry(input_frame)
capacity_entry.grid(row=2, column=1, padx=5, pady=5)

tk.Label(input_frame, text="Year:").grid(row=3, column=0, padx=5, pady=5, sticky="e")
year_entry = tk.Entry(input_frame)
year_entry.grid(row=3, column=1, padx=5, pady=5)

tk.Label(input_frame, text="Color:").grid(row=4, column=0, padx=5, pady=5, sticky="e")
color_var = tk.StringVar(value="White")
color_menu = ttk.Combobox(input_frame, textvariable=color_var)
color_menu['values'] = ("White", "Blue", "Silver", "Gray", "Red")
color_menu.grid(row=4, column=1, padx=5, pady=5)

# Buttons
btn_frame = tk.Frame(input_frame)
btn_frame.grid(row=5, column=0, columnspan=2, pady=10)
tk.Button(btn_frame, text="Add", width=10, command=add_airplane).pack(side="left", padx=5)
tk.Button(btn_frame, text="Update", width=10, command=update_airplane).pack(side="left", padx=5)
tk.Button(btn_frame, text="Delete", width=10, command=delete_airplane).pack(side="left", padx=5)
tk.Button(btn_frame, text="Clear", width=10, command=clear_inputs).pack(side="left", padx=5)

# --------- Display Frame --------- #
display_frame = tk.LabelFrame(root, text="Available Airplanes (1980–2035)")
display_frame.pack(fill="both", expand=True, padx=10, pady=10)

columns = ("ID", "Airline", "Model", "Capacity", "Year", "Color")
tree = ttk.Treeview(display_frame, columns=columns, show="headings")
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=120)
tree.pack(fill="both", expand=True)
tree.bind("<<TreeviewSelect>>", on_tree_select)

# --------- Theme Switcher --------- #
theme_frame = tk.LabelFrame(root, text="Theme", padx=10, pady=10)
theme_frame.pack(fill="x", padx=10, pady=5)

tk.Button(theme_frame, text="Light", command=lambda: apply_theme("light")).pack(side="left", padx=5)
tk.Button(theme_frame, text="Dark", command=lambda: apply_theme("dark")).pack(side="left", padx=5)
tk.Button(theme_frame, text="RGB", command=lambda: apply_theme("rgb")).pack(side="left", padx=5)

# --------- Initialize --------- #
apply_theme("light")
view_airplanes()
root.mainloop()

# Close DB connection on exit
conn.close()

TclError: no display name and no $DISPLAY environment variable